<a href="https://colab.research.google.com/github/R3gm/Colab-resources/blob/main/Video_generation_Interpolation_PyTTI_Tools__FiLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FiLM: Frame Interpolation for Large Motion

This notebook is a convenience wrapper around a pre-trained model released by ML researchers at Google. Learn more about their work here: 

* https://github.com/google-research/frame-interpolation

---

Notebook authored by David Marx ([@DigThatData](https://twitter.com/DigThatData)), released under the MIT license.

If you experience any issue with or have suggestions regarding this notebook, please report them here:
* https://github.com/pytti-tools/pytti-notebook/issues/new 


In [ ]:
# This top bit only neneds to be run once
# ...may as well set up the notebook to 
# assume the user is only going to run all
# of setup once.
from pathlib import Path
import os

drive_mounted = False
gdrive_fpath = '.'
local_path = '/content/'

##############################

#@markdown # 1. Setup Workspace

#@markdown Run this cell to perform setup.

#@markdown Mounting your google drive is optional.
#@markdown If you mount your drive, code and models will be downloaded to it.
#@markdown This should reduce setup time after your first run.

###################

# Optionally Mount GDrive

mount_gdrive = False # @param{type:"boolean"}
if mount_gdrive and not drive_mounted:
    from google.colab import drive

    gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
    gdrive_subdirectory = 'MyDrive/interpolation' #@param{type:"string"}
    gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
    try:
        drive.mount(gdrive_mountpoint, force_remount = True)
        !mkdir -p {gdrive_fpath}
        %cd {gdrive_fpath}
        local_path = gdrive_fpath
        drive_mounted = True
    except OSError:
        print(
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            "If you received a scary OSError and your drive"
            " was already mounted, ignore it."
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            )
        raise

#####################

# Perform rest of setup

if not Path('./frame-interpolation').exists():
    !git clone https://github.com/pytti-tools/frame-interpolation

try:
    import frame_interpolation
except ModuleNotFoundError:
    %pip install -r ./frame-interpolation/requirements_colab.txt
    %pip install ./frame-interpolation

#url = "https://drive.google.com/drive/folders/1GhVNBPq20X7eaMsesydQ774CgGcDGkc6?usp=sharing"
share_id = "1GhVNBPq20X7eaMsesydQ774CgGcDGkc6"

if not (Path(local_path) / 'saved_model').exists():
    !pip install --upgrade gdown
    !gdown --folder {share_id}

# create default frame
!mkdir -p frames

Cloning into 'frame-interpolation'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 179 (delta 57), reused 42 (delta 42), pack-reused 99
Receiving objects: 100% (179/179), 26.97 MiB | 26.20 MiB/s, done.
Resolving deltas: 100% (84/84), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 90.7 MB/s eta 0

In [ ]:
!pip install loguru apache_beam mediapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.3 MB/s eta 0:00:00


In [ ]:
#@markdown # 2. Setup Interpolation

#@markdown Specify the directory containing your video frames with the `frames_dir` parameter.

frames_dir = "frames" #@param{'type':'string'}

#@markdown A single pass of the interpolation procedure adds a frame between each contiguous pair of frames in `frames_dir`.

#@markdown If you start with $n$ frames in `frames_dir` and set `recursive_interpolation_passes` to $k$, your total number of frames
#@markdown after interpolation will be: 
#@markdown $$2^k (n-1) -1$$

recursive_interpolation_passes = 1 #@param{'type':'integer'}

#@markdown Check this box to generate a video output. If no output video will be generated, the FPS option can be ignored.
output_video = True #@param{'type':'boolean'}
output_video_fps = 30 #@param{'type':'number'}

In [ ]:
#@markdown # 3. Interpolate!
#@markdown Results will be saved in a subdirectory of `frames_dir` named "interpolated_frames"

from loguru import logger

logger.info("Beginning interpolation...")

if output_video:
  !python -m frame_interpolation.eval.interpolator_cli \
      --model_path ./saved_model \
      --pattern {frames_dir} \
      --fps {output_video_fps} \
      --output_video
      

else:
    !python -m frame_interpolation.eval.interpolator_cli \
      --model_path ./saved_model \
      --pattern {frames_dir} \

logger.info("Interpolation comlpete.")

2023-05-26 02:58:24.330 | INFO     | __main__:<cell line: 6>:6 - Beginning interpolation...


2023-05-26 02:58:25.002432: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 02:58:26.385591: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-26 02:58:28.756518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-26 02:58:28.793803: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

2023-05-26 02:59:16.822 | INFO     | __main__:<cell line: 14>:14 - Interpolation comlpete.


video in: /content/frames/interpolated.mp4